In [1]:
#Run with HoopBuddy kernel

import os
import glob
import io
import pprint
import tempfile
import matplotlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from six import BytesIO
from IPython import display
from urllib.request import urlopen
import numpy as np
from ultralytics import YOLO


In [2]:
#intersection code written by ChatGPT to save time
def on_segment(p, q, r):
    """
    Check if point q lies on line segment 'pr'
    """
    if (q[0] <= max(p[0], r[0]) and q[0] >= min(p[0], r[0]) and
            q[1] <= max(p[1], r[1]) and q[1] >= min(p[1], r[1])):
        return True
    return False

def orientation(p, q, r):
    """
    Find the orientation of the ordered triplet (p, q, r).
    Returns:
    0 -> p, q, and r are collinear
    1 -> Clockwise
    2 -> Counterclockwise
    """
    val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
    if val == 0:
        return 0  # Collinear
    return 1 if val > 0 else 2  # Clockwise or counterclockwise

def do_intersect(p1, q1, p2, q2):
    """
    Check if the line segment 'p1q1' and 'p2q2' intersect.
    """
    # Find the four orientations needed for the general and special cases
    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)

    # General case
    if o1 != o2 and o3 != o4:
        return True

    # Special cases
    # p1, q1, and p2 are collinear and p2 lies on segment p1q1
    if o1 == 0 and on_segment(p1, p2, q1):
        return True

    # p1, q1, and q2 are collinear and q2 lies on segment p1q1
    if o2 == 0 and on_segment(p1, q2, q1):
        return True

    # p2, q2, and p1 are collinear and p1 lies on segment p2q2
    if o3 == 0 and on_segment(p2, p1, q2):
        return True

    # p2, q2, and q1 are collinear and q1 lies on segment p2q2
    if o4 == 0 and on_segment(p2, q1, q2):
        return True

    # Otherwise, the segments don't intersect
    return False


In [18]:
def split_video_to_frames(video_path, output_dir, frames_per_second=10):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    video_capture = cv2.VideoCapture(video_path)
    video_fps = video_capture.get(cv2.CAP_PROP_FPS)
    frame_interval = int(video_fps // frames_per_second)
    frame_count = 0
    saved_frame_count = 0
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_dir, f'frame_{saved_frame_count:05d}.png')
            cv2.imwrite(frame_filename, frame)
            saved_frame_count += 1
        frame_count += 1
    video_capture.release()
    print(f"Saved {saved_frame_count} frames to {output_dir}")
split_video_to_frames('tests/166.mov', 'tests/split166') #path not setup in this GitHub, used locally for expirementation

Saved 24 frames to tests/split166


In [60]:
import math
import re

def crop_image_into_tiles_input_image(image, tile_size):
    image_width, image_height = image.size
    tiles_x = image_width // tile_size
    tiles_y = image_height // tile_size
    tiles = []
    for y in range(0, 2*tiles_y):
        for x in range(0, 2*tiles_x):
            left = x * (tile_size)/2
            upper = y * (tile_size)/2
            right = (x + 2) * (tile_size)/2
            lower = (y + 2) * (tile_size)/2
            tile = image.crop((left, upper, right, lower))
            tiles.append(tile)
    return tiles

def process_images_in_directory(directory_path):
    image_extensions = ['*.png', '*.jpg', '*.jpeg']
    image_files = []
    for extension in image_extensions:
        image_files.extend(glob.glob(os.path.join(directory_path, extension)))
    return image_files

model_hoop = YOLO('../models/Hoop Detection/weights/best.pt')
model_ball = YOLO('../models/Ball Detection/weights/best.pt')
model_key = YOLO('../models/Pose Key/weights/best.pt')
directory_path = 'tests/split166/' #used locally for expirementation
directory_visuals = 'visuals/'
image_files = process_images_in_directory(directory_path)
image_files.sort(
    key=lambda x: int(re.search(r'(\d+)', os.path.basename(x)).group())  # Extract the number for sorting
)
tile_size = 400 
ballHistory = [0, 0]
hoopHistory = [0, 0, 10]
counter = 0

def calculate_distance(x1, y1, x2, y2):
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return distance

for frameNum, frame in enumerate(image_files):
    image_path = frame
    image = Image.open(image_path) 
    original_image = image
    frameName = frame.split("/")[-1].split(".png")[0]
    keyPrediction = model_key(image)

    x1 = 0
    x2 = 0
    y1 = 0
    y2 = 0

    # Ensure there's at least one prediction
    if keyPrediction and len(keyPrediction[0].keypoints) > 0  and len(keyPrediction[0].boxes) > 0:
        highest_confidence_prediction = max(keyPrediction[0], key=lambda kp: kp.boxes[0].conf[0]).keypoints
        
        keypoint_coords_charge = list(list(highest_confidence_prediction.xy[0].cpu().numpy())[5])
        keypoint_coords_bottom_block = list(list(highest_confidence_prediction.xy[0].cpu().numpy())[4])
        x1 = keypoint_coords_charge[0]-2*(keypoint_coords_charge[0]-keypoint_coords_bottom_block[0])
        y1 = 0
        x2 = keypoint_coords_charge[0]
        if x1 > x2:
            savedX1 = x1
            x1 = x2
            x2 = savedX1
        y2 = keypoint_coords_bottom_block[1]

    tileHoop = image.crop((x1, y1, x2, y2))

    translateXHoop = x1
    translateYHoop = y1

    if tileHoop.width > 0 and tileHoop.height > 0:
        hoopPrediction = model_hoop(tileHoop)
        detected_hoops = hoopPrediction[0].boxes
    
        potentialHoops = []
        if detected_hoops and len(detected_hoops) > 0:
            highest_confidence_hoop = max(detected_hoops, key=lambda box: box.conf[0])
            
            box = highest_confidence_hoop.xyxy[0].cpu().numpy().astype(int)
            x1, y1, x2, y2 = box
            x1 = x1 + translateXHoop
            y1 = y1 + translateYHoop
            x2 = x2 + translateXHoop
            y2 = y2 + translateYHoop
            potentialHoops.append([(x1+x2)/2, y1, x2-x1])

        image = tileHoop
        image_width, image_height = image.size
        tiles_x = image_width // tile_size
        tiles_y = image_height // tile_size
        tiles = crop_image_into_tiles_input_image(image, tile_size)
        
        potentialBalls = []
        for i, tile in enumerate(tiles):
            ballPrediction = model_ball(tile)
            detected_balls = ballPrediction[0].boxes 
            for ball in detected_balls:
                confidence = ball.conf[0]
                if confidence > 0.5: 
                    box = ball.xyxy[0].cpu().numpy().astype(int)
                    x1, y1, x2, y2 = box
                    potentialBalls.append([(x1+x2)/2 + (i%(tiles_x*2)) * (tile_size)/2 + translateXHoop, y1 + (i//(tiles_x*2)) * (tile_size)/2 + translateYHoop]) #add tile offset
    
        bestNewBall = [0, 0]
        bestBenchMarkDistance = 9999999999
        for newBall in potentialBalls:
            distance = calculate_distance(ballHistory[0], ballHistory[1], newBall[0], newBall[1])
            if distance < bestBenchMarkDistance:
                bestBenchMarkDistance = distance
                bestNewBall = [newBall[0], newBall[1]]

        bestNewHoop = potentialHoops[0]
        
        if do_intersect((bestNewBall[0], bestNewBall[1]), (ballHistory[0], ballHistory[1]), (bestNewHoop[0]-bestNewHoop[2]/2, bestNewHoop[1]), (bestNewHoop[0]+bestNewHoop[2]/2, bestNewHoop[1])) or do_intersect((bestNewBall[0], bestNewBall[1]), (ballHistory[0], ballHistory[1]), (hoopHistory[0]-hoopHistory[2]/2, hoopHistory[1]), (hoopHistory[0]+hoopHistory[2]/2, hoopHistory[1])) == True:
            counter = counter + 1
        
        ballHistory = bestNewBall
        hoopHistory = bestNewHoop

        annotation_directory = "annotated_visuals"
        plt.figure()
        plt.imshow(original_image)
        plt.scatter([bestNewBall[0]], [bestNewBall[1]], c='red', marker='x', s=100)  # Plot the "X" mark
        plt.scatter([bestNewHoop[0]], [bestNewHoop[1]], c='green', marker='x', s=100)  # Plot the "X" mark
        plt.axis('off')
        output_path_annotation = os.path.join(annotation_directory, os.path.basename(str(frameNum)))
        plt.savefig(output_path_annotation, bbox_inches='tight', pad_inches=0)
        plt.close()
    else:
        print("No key detected")
    

print(counter) #counts number of hoops made, but this code could be easily tweaked to timestamp on longer videos and log to a file when a basket is made


0: 576x1024 1 key, 198.7ms
Speed: 4.9ms preprocess, 198.7ms inference, 2.4ms postprocess per image at shape (1, 3, 576, 1024)

0: 256x192 1 hoop, 15.9ms
Speed: 0.5ms preprocess, 15.9ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 192)

0: 416x416 (no detections), 50.0ms
Speed: 1.0ms preprocess, 50.0ms inference, 0.2ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 51.5ms
Speed: 0.9ms preprocess, 51.5ms inference, 0.2ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 ball, 50.5ms
Speed: 0.8ms preprocess, 50.5ms inference, 0.3ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 53.3ms
Speed: 0.7ms preprocess, 53.3ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 52.9ms
Speed: 0.8ms preprocess, 52.9ms inference, 0.2ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 49.5ms
Speed: 0.8ms preprocess, 49.5ms inference, 0.2ms postproc